In [ ]:
from ultralytics import YOLO
import cv2
import os
import csv
from datetime import datetime

# Charger ton modèle YOLOv11
# Tu peux mettre "yolov11.pt" ou ton modèle personnalisé "runs/detect/train/weights/best.pt"
model = YOLO("yolov11.pt")

# Création du dossier de sortie
output_dir = "data/detections"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV de comptage
csv_file = os.path.join(output_dir, "comptage_pneus.csv")
if not os.path.exists(csv_file):
    with open(csv_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["datetime", "nb_pneus_detectes", "nom_image"])

# Ouvrir la webcam (0 pour la caméra principale)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Erreur : impossible d’ouvrir la caméra.")
    exit()

print("Appuyez sur Q pour quitter...")

# Boucle de capture
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Lancer la détection avec le mode tracking activé
    results = model.track(frame, persist=True, verbose=False)

    pneu_count = 0

    # Parcourir les résultats pour tracer les cadres
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for box in boxes:
                cls = int(box.cls[0])
                label = model.names[cls]
                if "pneu" in label.lower() or "tire" in label.lower():
                # if label.lower() == "pneu":  # à utiliser si on entraine son propre modèle avec une classe "pneu"
                    pneu_count += 1
                    # Récupérer coordonnées
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    track_id = int(box.id[0]) if box.id is not None else 0

                    # Dessiner le cadre et l'ID sur l'image
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label} #{track_id}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Afficher le nombre de pneus détectés
    cv2.putText(frame, f"Pneus détectés: {pneu_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Montrer la vidéo avec détection
    cv2.imshow("YOLOv11 - Comptage & Tracking pneus", frame)

    # Sauvegarde image + log CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    img_name = f"{timestamp}.jpg"
    img_path = os.path.join(output_dir, img_name)
    cv2.imwrite(img_path, frame)

    with open(csv_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([timestamp, pneu_count, img_name])

    # Sortir avec Q
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()